# other_cbs

blah blah

In [3]:
#| defult_export other_cbs

In [4]:
from pw.core import *
from pw.crawler import *
from pw.helper import *

In [5]:
def screener_url(t): return f"https://www.screener.in/company/{t}/consolidated/"

In [6]:
#| export
class AnualReportCB(Callback):
    """
    Callback helping traveling all the links available in the same domain.
    """
    def __init__(self, tickets):
        self.order = 1
        self.links = {t : None for t in tickets}


    async def  after_visit(self, crawler, idx):
        
        pg = crawler.pages[idx]
        await pg.wait(10000)
        show_more_ele = await pg.find_ele('//button[@class="a show-more-button"]')
        for i in show_more_ele:
            await i.left_click()
        #await pg.pause()
        
        annual_reports = await pg.find_ele('//div[@class="documents annual-reports flex-column"]//child::a')

        links = {(await tag.get_text()).split("from")[0].strip() : await tag.get_attribute("href") 
                      for tag in annual_reports if await tag.get_attribute("href")}
        

        for i in list(self.links.keys()):
            if i in pg.url:
                break
        self.links[i] = links


In [7]:
t  = ["TATAMOTORS"]
url = [screener_url(i) for i in t]
C = Crawl(1, url, [AnualReportCB(t)])
await C.run(headless=False, stealth= True,devtools=True )

In [ ]:
C.cbs[0].links

In [ ]:
class ConcallsCB(Callback):
    """
    Callback helping traveling all the links available in the same domain.
    """
    def __init__(self, tickets):
        self.order = 2
        self.trans_script = {t : None for t in tickets}
        
    async def  after_visit(self, crawler, idx):
        
        pg = crawler.pages[idx]
        show_more_ele = await pg.find_ele('//button[@class="a show-more-button"]')
        for i in show_more_ele:
            await i.left_click()
        
        n = len(await pg.find_ele('//div[@class="documents concalls flex-column"]//child::li'))
        dic = {}

        for i in range(1, n+1):
            label = await pg.find_ele(f'//div[@class="documents concalls flex-column"]//child::li[{i}]//div[1]')
            label = (await label[0].get_text()).strip()

            trans = await pg.find_ele(f'//div[@class="documents concalls flex-column"]//child::li[{i}]//a[contains(text(), "Transcript")]')

            if trans is None: # fetch ppt if Transcript is not presnet
                trans = await pg.find_ele(f'//div[@class="documents concalls flex-column"]//child::li[{i}]//a[contains(text(), "PPT")]') 
            #print(label, trans)

            dic[label] = await trans[0].get_attribute("href")

            print(dic)
        
        for i in list(self.trans_script.keys()):
            if i in pg.url:
                break
        self.trans_script[i] = dic

In [ ]:
t  = ["TATAMOTORS", "ASHOKLEY"]
url = [screener_url(i) for i in t]
C = Crawl(2, url, [AnualReportCB(t), ConcallsCB(t)])
await C.run(headless=False, stealth= True,devtools=True )
#C.cbs[1].trans_script['TATAMOTORS']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()